![Rigetti logo](https://qcs.rigetti.com/static/img/rigetti-logo.svg)

&copy; Copyright 2022-2024 Rigetti & Co, Inc.

*This Computer Software is developed under Agreement HR00112230006 "Guidestars for Modular Quantum Computing" between Rigetti & Co LLC and the Defense Advanced Research Projects Agency (DARPA). Use, duplication, or disclosure is subject to the restrictions as stated in Agreement HR00112230006 between the Government and the Performer.*

# `rigetti-resource-estimation` basic demo

### 1. Some global imports and setup

In [ ]:
from pathlib import Path

from rigetti_resource_estimation.estimation_pipeline import estimation_pipeline

input_dir = Path("./input")

output_dir = Path("./output")
output_dir.mkdir(parents=True, exist_ok=True)

### 2. Input logical circuits (in OpenQASM2.0)

Simple Quantum Fourier Transform circuits with 4 and 10 qubits.

In [ ]:
circ_path_qft4 = input_dir / "qft4.qasm"

In [ ]:
circ_path_qft10 = input_dir / "qft10.qasm"

### 3. Hardware configuration parameters
All hardware, architectural, algorithmic configurations are unified and set in `python/params.yaml`, which can be editted as desired. A description of some of these parameters and their default values are listed here for convenience.

* ```phys_gate_error_rate: 0.001```

  A homogeneous gate error rate that sets how quickly logical errors are suppressed. For example, 1e-3
  means a factor of 10 suppression with each increase of $d$ by 2 here; this is the value we adopt.

* ```qcycle_char_timescale_ns: 50```

  Characteristic quantum timescale for the hardware architecture to execute physical-level gates in ns.
  This quantity specifies the cycle timescale for surface code's lattice surgery ops; we adopt a nominal value of 50ns.

* ```target_p_algo: 0.5```

  Desired algorithm failure probability/threshold/rate budget targeted for executing the full logical circuit.
  
* ```1q_unitary_decomp_method: gridsynth```

  Decomposition method for transforming arbitrary (small-angle) 1Q unitaries into Clifford+T at measurement
  points. This choice will set two constants used in the decomposition formula of c0*log2(1/eps)+c1. 
  --- Supported inputs ---
    "gridsynth": the original GridSynth decomposition proposed by [Ross and Selinger 2014, arXiv:1403.2975]
    and is used in upstream tools such as Jabalizer and NewSynth. Here, c0=4.0 and c1=0.
    "mixed_fallback": the state-of-the-art mixed-fallback approach improving on the GridSynth's Clifford+T
    results published by [Kliuchnikov et al. 2022, arXiv:2203.10064] among other decomposition methods. Here,
    c0=0.57 and c1=8.83. (DEFAULT)

* ```fixed_eps: null```

  If set here, manually, it will override the automated calculation of `eps` based on transcendental equations
  with this fixed value.

### 4. Running the estimation experiment

In [ ]:
output_csv = output_dir / "test.csv"

Resource estimation through graph-state-based compilation:

In [ ]:
estimation_pipeline(
    circ_path=str(circ_path_qft4),
    est_method="jabalizer",
    output_csv=output_csv,
    log="DEBUG",
)

Resource estimation through T-counting methods:

In [ ]:
estimation_pipeline(
    circ_path=str(circ_path_qft4),
    est_method="t_counting",
    graph_state_opt="no_compile",
    output_csv=output_csv,
    log="INFO",
)